In [1]:
#pip install selenium
from time import sleep
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import re
import numpy as np
from tqdm import tqdm

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.chrome.service import Service

In [2]:
#pip install --upgrade selenium

In [3]:
chrome_options = webdriver.ChromeOptions()
# Add any additional options you need

service = Service(executable_path="C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe")

# Specify the binary location (replace with your Chrome executable path)
chrome_options.binary_location = "C:\\Program Files (x86)\\Google\\Chrome\\Application\\chrome.exe"

# Launch the driver
#driver = webdriver.Chrome(service=service, options=chrome_options)
driver = webdriver.Chrome()

In [4]:
# get user input for routes
sources = []
destinations = []
print("Please enter -1 when done.")
print("-"*10)
while True:
    sources.append(input("From which city?\n"))
    if "-1" in sources: 
        sources.pop(-1)
        break
    destinations.append(input("Where to?\n"))
    if "-1" in destinations: 
        sources.pop(-1)
        destinations.pop(-1)
        break
    print("-"*10)

print("\nRoutes:")
for i in range(len(sources)):
    print(f"{sources[i]} => {destinations[i]}")

Please enter -1 when done.
----------
From which city?
DUB
Where to?
MAD
----------
From which city?
MAD
Where to?
DUB
----------
From which city?
DUB
Where to?
TFN
----------
From which city?
TFN
Where to?
DUB
----------
From which city?
DUB
Where to?
IBZ
----------
From which city?
IBZ
Where to?
DUB
----------
From which city?
-1

Routes:
DUB => MAD
MAD => DUB
DUB => TFN
TFN => DUB
DUB => IBZ
IBZ => DUB


In [5]:
# get user input for period (start and end date)
start_date = np.datetime64(input('Start Date, Please use YYYY-MM-DD format only '))
end_date = np.datetime64(input('End Date, Please use YYYY-MM-DD format only '))
days = end_date - start_date
num_days = days.item().days

Start Date, Please use YYYY-MM-DD format only 2023-12-15
End Date, Please use YYYY-MM-DD format only 2023-12-15


In [6]:
def get_airlines(soup):
    airline = []
    airlines = soup.find_all('div',class_='J0g6-operator-text',text=True)
    for i in airlines:
        airline.append(i.text)
    return airline
    
def get_total_stops(soup):
    stops_list = []
    stops = soup.find_all('span',class_='JWEO-stops-text')

    for i in stops:
        stops_list.append(i.text)
    return stops_list

def get_price(soup):
    prices = []
    price = soup.find_all('div',class_='f8F1-price-text')

    for i in price:
        prices.append(i.text)
    return prices

def get_duration(soup):
    duration_list = []
    duration = soup.find_all('div' , class_='xdW8 xdW8-mod-full-airport')
    for i in duration:
        duration_list.append(i.text)
    return duration_list

In [7]:
# Set the path to the ChromeDriver executable
chromedriver_path = "C:\\Users\\eduma\\Downloads\\chromedriver-win64\\chromedriver-win64\\chromedriver.exe"

# Launch the driver using the Service class
service = Service(chromedriver_path)
#driver = webdriver.Chrome(service=service)
#driver = webdriver.Chrome()

for i in range(len(sources)):
    column_names = ["Airline", "Source", "Destination", "Duration", "Total stops", "Price", "Date"]
    df = pd.DataFrame(columns=column_names)

    for j in tqdm(range(num_days + 1)):
        # close and open driver every 10 days to avoid captcha
        if j % 10 == 0:
            driver.quit()
            #service = Service(chromedriver_path)
            #driver = webdriver.Chrome(service=service)
            service = Service()
            driver = webdriver.Chrome()

        url = f"https://www.kayak.ie/flights/{sources[i]}-{destinations[i]}/{start_date+j}"
        driver.get(url)
        sleep(15)

        # click show more button to get all flights
        try:
            show_more_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.XPATH, '//div[@class="ULvh-button show-more-button"]'))
            )
            show_more_button.click()
            print("Button pressed")
        except NoSuchElementException:
            print("Show more button not found. Skipping to the next iteration.")
            continue

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        airlines = get_airlines(soup)
        total_stops = get_total_stops(soup)
        prices = get_price(soup)
        duration = get_duration(soup)
        print(airlines)
        print(total_stops)
        print(prices)
        print(duration)
        new_data = pd.DataFrame({
            'Airline': airlines,
            'Duration': duration,
            'Total stops': total_stops,
            'Price': prices,
            'Date': start_date + j
        })
        df = pd.concat([df, new_data], ignore_index=True, axis=0)

    df['Source'] = sources[i]
    df['Destination'] = destinations[i]
    df = df.replace('\n', '', regex=True)
    df = df.reset_index(drop=True)

    # save data as a CSV file for each route
    df.to_csv(f'{sources[i]}_{destinations[i]}.csv', index=False)
    print(f"Successfully saved {sources[i]} => {destinations[i]} route as {sources[i]}_{destinations[i]}.csv")

driver.quit()

  0%|          | 0/1 [00:00<?, ?it/s]

Button pressed


100%|██████████| 1/1 [00:26<00:00, 26.18s/it]


['Vueling', 'Vueling', 'Iberia Express', 'Iberia', 'Aer Lingus', 'Aer Lingus', 'Iberia', 'Aer Lingus, Air Europa', 'KLM', 'Lufthansa', 'Transavia France, Air Europa', 'Vueling, Air Europa', 'Vueling, Air Europa', 'Vueling, Iberia', 'British Airways']
['direct', '1 stop', 'direct', 'direct', 'direct', 'direct', 'direct', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€167', '€107', '€172', '€175', '€188', '€199', '€210', '€131', '€388', '€323', '€250', '€262', '€316', '€337', '€822']
['2h 35m', '5h 25m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '2h 35m', '6h 10m', '5h 05m', '5h 55m', '6h 20m', '6h 20m', '5h 25m', '5h 55m', '4h 40m']
Successfully saved DUB => MAD route as DUB_MAD.csv


  0%|          | 0/1 [00:00<?, ?it/s]

...

['Vueling', 'Vueling', 'Aer Lingus, Vueling', 'Iberia', 'Vueling', 'Transavia France, Iberia', 'Iberia, Iberia Express', 'Iberia', 'British Airways', 'Vueling, Iberia Express', 'British Airways', 'British Airways', 'Vueling, Iberia', 'British Airways', 'Vueling, Air Europa']
['1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '1 stop', '2 stops', '1 stop', '1 stop', '2 stops', '1 stop', '2 stops']
['€127', '€84', '€276', '€140', '€84', '€250', '€224', '€243', '€335', '€157', '€344', '€365', '€161', '€354', '€177']
['6h 20m', '17h 45m', '5h 05m', '12h 30m', '20h 45m', '7h 50m', '12h 30m', '15h 15m', '12h 20m', '15h 55m', '12h 15m', '12h 35m', '15h 55m', '13h 15m', '15h 35m']
Successfully saved DUB => IBZ route as DUB_IBZ.csv


  0%|          | 0/1 [00:00<?, ?it/s]

Button pressed


100%|██████████| 1/1 [00:25<00:00, 25.85s/it]

['Vueling', 'Vueling', 'Vueling, Transavia France', 'Vueling, Aer Lingus', 'British Airways', 'British Airways', 'Vueling, Aer Lingus', 'Iberia, British Airways', 'Iberia', 'Iberia, British Airways', 'British Airways', 'Vueling, Aer Lingus', 'Iberia', 'British Airways', 'Iberia']
['1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop', '1 stop']
['€204', '€95', '€188', '€263', '€336', '€324', '€272', '€353', '€550', '€340', '€336', '€272', '€526', '€321', '€509']
['4h 50m', '18h 40m', '5h 30m', '6h 10m', '7h 45m', '8h 25m', '8h 20m', '7h 45m', '4h 50m', '8h 25m', '8h 45m', '9h 35m', '5h 30m', '9h 45m', '5h 55m']
Successfully saved IBZ => DUB route as IBZ_DUB.csv
